In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('cleaned_data.csv')
data ['Date'] = data['Unnamed: 0']
data.set_index('Date', inplace=True)
data.drop(columns=['Unnamed: 0'], inplace=True)
data

,exdate,cp_flag,strike_price,best_bid,best_offer,volume,impl_volatility,delta,gamma,optionid,contract_size,D to Expiration,Close,Moneyness
Date,,,,,,,,,,,,,,
2000-01-03,2000-01-22,P,1505.0,55.250,57.250,0,0.212891,-0.734362,0.004767,10000760,100,19,1455.219971,1.034208
2000-01-03,2000-01-22,P,1545.0,89.625,91.625,0,0.233571,-0.858130,0.002975,10007866,100,19,1455.219971,1.061695
2000-01-03,2000-01-22,C,1535.0,2.000,2.750,77,0.170048,0.093890,0.003049,10036858,100,19,1455.219971,1.054823
2000-01-03,2000-01-22,P,1540.0,84.875,86.875,0,0.227691,-0.850098,0.003167,10056576,100,19,1455.219971,1.058259
2000-01-03,2000-01-22,P,1535.0,80.375,82.375,0,0.224673,-0.838002,0.003378,10084875,100,19,1455.219971,1.054823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-31,2023-10-06,P,4780.0,248.100,254.000,0,0.100853,-0.953237,0.000685,156675899,100,36,4507.660156,1.060417
2023-08-31,2023-10-06,P,4800.0,267.800,272.900,0,0.102260,-0.962751,0.000561,156187029,100,36,4507.660156,1.064854
2023-08-31,2023-10-06,P,4825.0,292.100,297.200,0,0.103956,-0.972125,0.000435,156545972,100,36,4507.660156,1.070400


In [3]:
data[data['optionid'] == 10000760]

,exdate,cp_flag,strike_price,best_bid,best_offer,volume,impl_volatility,delta,gamma,optionid,contract_size,D to Expiration,Close,Moneyness
Date,,,,,,,,,,,,,,
2000-01-03,2000-01-22,P,1505.0,55.250,57.250,0,0.212891,-0.734362,0.004767,10000760,100,19,1455.219971,1.034208
2000-01-04,2000-01-22,P,1505.0,104.500,106.500,0,0.257080,-0.891543,0.002397,10000760,100,18,1399.420044,1.075446
2000-01-05,2000-01-22,P,1505.0,101.750,103.750,0,0.254144,-0.895625,0.002427,10000760,100,17,1402.109985,1.073382
2000-01-06,2000-01-22,P,1505.0,111.750,113.750,0,0.416411,-0.775640,0.002527,10000760,100,16,1403.449951,1.072357
2000-01-07,2000-01-22,P,1505.0,56.500,58.500,0,NaN,NaN,NaN,10000760,100,15,1441.469971,1.044073
2000-01-10,2000-01-22,P,1505.0,43.375,45.375,0,NaN,NaN,NaN,10000760,100,12,1457.599976,1.032519
2000-01-11,2000-01-22,P,1505.0,61.250,63.250,0,NaN,NaN,NaN,10000760,100,11,1438.560059,1.046185
2000-01-12,2000-01-22,P,1505.0,73.000,75.000,0,0.256997,-0.879527,0.003471,10000760,100,10,1432.250000,1.050794
2000-01-13,2000-01-22,P,1505.0,56.750,58.750,0,0.241790,-0.839749,0.004694,10000760,100,9,1449.680054,1.038160


In [4]:
data['D to Expiration'].unique()

array([19, 18, 17, 16, 15, 12, 11, 39, 10, 38,  9, 37,  8, 36,  4, 32,  3,
       31,  2, 30, 29, 26, 25, 24, 23, 22, 33,  5,  1,  7,  0,  6, 35, 28,
       21, 14, 34, 27, 20, 13], dtype=int64)

In [9]:
def filter_data(df):
    result = pd.DataFrame()
    selected_optionids = set()
    
    for date, group in df.groupby(df.index):
        # Separate calls and puts
        calls = group[group['cp_flag'] == 'C']
        puts = group[group['cp_flag'] == 'P']
        
        # Define maturity groups
        maturity_groups = {
            '1_day': (0, 3),
            '1_week': (4, 14),
            '1_month': (23, 37)
        }
        
        # Find the closest moneyness to 1 for each maturity group
        for group_name, (min_days, max_days) in maturity_groups.items():
            # Filter options within the maturity group
            available_calls = calls[(calls['D to Expiration'] >= min_days) & (calls['D to Expiration'] <= max_days)]
            available_puts = puts[(puts['D to Expiration'] >= min_days) & (puts['D to Expiration'] <= max_days)]
            
            if not available_calls.empty:
                # Select the call with moneyness closest to 1
                closest_call = available_calls.iloc[(available_calls['Moneyness'] - 1).abs().argsort()[:1]]
                selected_optionids.add(closest_call['optionid'].values[0])
            
            if not available_puts.empty:
                # Select the put with moneyness closest to 1
                closest_put = available_puts.iloc[(available_puts['Moneyness'] - 1).abs().argsort()[:1]]
                selected_optionids.add(closest_put['optionid'].values[0])
    
    # Include all rows related to the selected optionids
    result = df[df['optionid'].isin(selected_optionids)]
    
    return result

# Apply the filter function
filtered_data = filter_data(data)

In [10]:
filtered_data

,exdate,cp_flag,strike_price,best_bid,best_offer,volume,impl_volatility,delta,gamma,optionid,contract_size,D to Expiration,Close,Moneyness
Date,,,,,,,,,,,,,,
2000-01-03,2000-01-22,P,1465.0,30.000,32.000,68,0.218037,-0.521459,0.005654,10526329,100,19,1455.219971,1.006721
2000-01-03,2000-01-22,P,1440.0,19.875,21.125,691,0.227943,-0.385638,0.005192,10563021,100,19,1455.219971,0.989541
2000-01-03,2000-01-22,C,1460.0,23.375,25.375,386,0.191068,0.505087,0.006461,10949021,100,19,1455.219971,1.003285
2000-01-03,2000-01-22,C,1450.0,29.125,32.000,1312,0.198929,0.567313,0.006117,11070856,100,19,1455.219971,0.996413
2000-01-03,2000-01-22,C,1465.0,20.625,22.625,0,0.187986,0.472246,0.006551,11274505,100,19,1455.219971,1.006721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-31,2023-09-29,P,4435.0,28.800,29.100,21,0.128259,-0.281094,0.002070,154635934,100,29,4507.660156,0.983881
2023-08-31,2023-10-02,C,4515.0,65.400,65.900,0,0.111276,0.540238,0.002672,156631215,100,32,4507.660156,1.001628
2023-08-31,2023-10-02,P,4515.0,53.500,53.900,12,0.112415,-0.459972,0.002645,156631264,100,32,4507.660156,1.001628


In [11]:
filtered_data[filtered_data['optionid'] == 10949021]

,exdate,cp_flag,strike_price,best_bid,best_offer,volume,impl_volatility,delta,gamma,optionid,contract_size,D to Expiration,Close,Moneyness
Date,,,,,,,,,,,,,,
2000-01-03,2000-01-22,C,1460.0,23.375,25.375,386,0.191068,0.505087,0.006461,10949021,100,19,1455.219971,1.003285
2000-01-04,2000-01-22,C,1460.0,6.250,7.250,357,0.209521,0.196236,0.004374,10949021,100,18,1399.420044,1.043289
2000-01-05,2000-01-22,C,1460.0,5.625,6.625,270,0.202314,0.190889,0.004582,10949021,100,17,1402.109985,1.041288
2000-01-06,2000-01-22,C,1460.0,3.375,4.125,1033,0.173372,0.149263,0.004711,10949021,100,16,1403.449951,1.040294
2000-01-07,2000-01-22,C,1460.0,15.250,16.500,1136,0.200727,0.401451,0.006824,10949021,100,15,1441.469971,1.012855
2000-01-10,2000-01-22,C,1460.0,20.500,22.500,267,0.212445,0.508188,0.007420,10949021,100,12,1457.599976,1.001647
2000-01-11,2000-01-22,C,1460.0,10.125,11.375,370,0.196550,0.348171,0.007899,10949021,100,11,1438.560059,1.014904
2000-01-12,2000-01-22,C,1460.0,5.250,6.250,284,0.168823,0.255452,0.008465,10949021,100,10,1432.250000,1.019375
2000-01-13,2000-01-22,C,1460.0,9.125,10.125,768,0.156621,0.404977,0.011530,10949021,100,9,1449.680054,1.007119


In [12]:
filtered_data.to_csv('filtered_data.csv')